# T2.8 Extraer con selenium datos de licitacións do Concello da Coruña
## Ralphy Núñez Mercado

Queremos facer un código que garde nunha BBDD a Microsoft SQL Server a información dos expedientes de licitación do Concello da Coruña e nun directorio (no actual) as capturas de pantalla de cada expediente.

Para montar o Microsoft SQL Server sigue a guía de: https://jfsanchez.es/docs/docker-5-mssql-server/

Debes empregar selenium e a páxina: https://contrataciondelestado.es

Debes navegar por ela coma se indica no documento.

Por cada elemento (expediente), meterse nel e facer captura de pantalla (automáticamente, coa API de selenium).

Entrega:

O código nun jupyter notebook.
A BBDD exportada de Microsoft SQL Server.
Un zip coas capturas de pantalla que fixo selenium.

### ⬇️ Imports 

In [9]:
import argparse
from enum import Enum
import sys
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select
import time
import random

### Definir Funciones

In [ ]:
def waitFinishLoad(driver):
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "page-loader")))
    WebDriverWait(driver, 10).until(EC.invisibility_of_element((By.ID, "page-loader")))
    alea=random.choice([0.5, 0.1, 0.2, 0.3, 0.4, 0.6])
    time.sleep(alea)

def clickSelectByPartialText(auxText, driver):
    waitFinishLoad(driver)
    driver.find_element_by_xpath("//select//option[contains(., '"+auxText+"')]").click()
    waitFinishLoad(driver)

def getCurrentPage(driver):
    print()

def getLastPage(driver):
    print()

def isThisLastPage(driver):
    if (getLastPage(driver) == getCurrentPage(driver)):
        return True
    return False

def goNextPage(driver,DOMINIO):
    driver.find_element(by=By.ID, value="id_domain").send_keys(DOMINIO)

    elem = driver.find_element(By.ID, "loginBtn")
    elem.click()

def clickByPartialText(auxText, driver):
    waitFinishLoad(driver)
    elem = driver.find_element_by_partial_link_text(auxText)
    elem.click()
    waitFinishLoad(driver)
    

### HECHAS POR MI
def select_opcion_portal(driver,link):
    driver.find_element(By.XPATH, f"//div/p/a[contains(@href,'{link}')]").click()

def click_opciones_busqueda(driver,opcion):
    driver.find_element(By.XPATH, f"//div/a/p/img[contains(@id,'{opcion}')]").click()
    
def buscar_by_id(driver,id):
    driver.find_element(By.ID,id).click()

def buscar_elementos_lista(driver, elemento, n=None):
    if n is not None:
        driver.find_element(By.XPATH, f"(//td[@class='tafelTreecanevas']/div[text()='A Coruña'])[{n}]").click()
    else:
        driver.find_element(By.XPATH, f"//td[@class='multiline'][following-sibling::td[2]/div[text()='{elemento}']]").click()

def select_combo_organo(driver,combo,valor):
    driver.find_element(By.XPATH, f"//select[contains(@id,'{combo}')]//option[@value='{valor}']").click()

### Definir Variables

In [11]:

URL = "https://contrataciondelestado.es/wps/portal/plataforma"

lista_tree = ['ENTIDADES LOCALES','Galicia','A Coruña','Ayuntamientos','A Coruña']

links = []

In [12]:
driver = webdriver.Firefox()

driver.get(URL)

select_opcion_portal(driver,'/wps/portal/licitaciones') # click en publicaciones

click_opciones_busqueda(driver,'viewns_Z7_AVEQAI930OBRD02JPMTPG21004_:form1:logoFormularioBusqueda') # click en licitaciones

buscar_by_id(driver,'viewns_Z7_AVEQAI930OBRD02JPMTPG21004_:form1:textBusquedaAvanzada') # click en busqueda avanzada

buscar_by_id(driver,'viewns_Z7_AVEQAI930OBRD02JPMTPG21004_:form1:idSeleccionarOCLink') # click en selecionar (organizacion contratante)

time.sleep(2) # test

for i, elemento in enumerate(lista_tree): # iterar sobre la lista para buscar los datos que queremos obtener
    if i == len(lista_tree) - 1:
        buscar_elementos_lista(driver, elemento, 2)
    else:
        buscar_elementos_lista(driver, elemento)

    time.sleep(0.5) # test


select_combo_organo(driver,'viewns_Z7_AVEQAI930OBRD02JPMTPG21004_:form1:comboNombreOrgano','93655277') # selecionar opcion combo organo

buscar_by_id(driver,'viewns_Z7_AVEQAI930OBRD02JPMTPG21004_:form1:botonAnadirMostrarPopUpArbolEO') # click en boton añadir
buscar_by_id(driver,'viewns_Z7_AVEQAI930OBRD02JPMTPG21004_:form1:button1') # click en buscar


n_paginas = driver.find_element(By.XPATH,'//*[@id="viewns_Z7_AVEQAI930OBRD02JPMTPG21004_:form1:textfooterInfoTotalPaginaMAQ"]').text

for i in range(int(n_paginas)-1):
    print(i)
    path_links = driver.find_elements(By.XPATH,"//table[contains(@id,'myTablaBusquedaCustom')]//a[2]")

    links.append([item.get_attribute("href") for item in path_links if item.get_attribute("href")])
    
    buscar_by_id(driver,'viewns_Z7_AVEQAI930OBRD02JPMTPG21004_:form1:footerSiguiente')
    time.sleep(0.2) # test


links_flat = list(itertools.chain(*links))

json.dumps(links_flat)

# driver.close()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


NoSuchWindowException: Message: Browsing context has been discarded
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:197:5
NoSuchWindowError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:706:5
assert.that/<@chrome://remote/content/shared/webdriver/Assert.sys.mjs:538:13
assert.open@chrome://remote/content/shared/webdriver/Assert.sys.mjs:147:4
GeckoDriver.prototype.findElements@chrome://remote/content/marionette/driver.sys.mjs:1879:15
despatch@chrome://remote/content/marionette/server.sys.mjs:318:40
execute@chrome://remote/content/marionette/server.sys.mjs:289:16
onPacket/<@chrome://remote/content/marionette/server.sys.mjs:262:20
onPacket@chrome://remote/content/marionette/server.sys.mjs:263:9
_onJSONObjectReady/<@chrome://remote/content/marionette/transport.sys.mjs:494:20


#### TODO
substituir los   
    ``` time.sleep ```   
por    
    ```WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, "//select[contains(@id,'viewns_Z7_AVEQAI930OBRD02JPMTPG21004_:form1:comboNombreOrgano')]"))) ```

- poner varibables para las funciones

- obtener enlaces

- obtener informacion de los enlaces